<a href="https://colab.research.google.com/github/Jonathan-code-hub/Many-Mini-OR-Problems/blob/main/LinearProgramming/LP_Problem_15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 15: Ricky’s Record Store now employs five full-time
employees and three part-time employees. The normal
workload is 40 hours per week for full-time and 20 hours
per week for part-time employees. Each full-time employee
is paid $6 per hour for work up to 40 hours per week and
can sell 5 records per hour. A full-time employee who works
overtime is paid \$10 per hour. Each part-time employee is
paid \$3 per hour and can sell 3 records per hour. It costs
Ricky \$6 to buy a record, and each record sells for \$9.
Ricky has weekly fixed expenses of \$500. He has established
the following weekly goals, listed in order of priority:
Goal 1 Sell at least 1,600 records per week.
Goal 2 Earn a profit of at least \$2,200 per week.
Goal 3 Full-time employees should work at most 100 hours
of overtime.
Goal 4 To increase their sense of job security, the number
of hours by which each full-time employee fails to work 40
hours should be minimized.
Formulate a preemptive goal programming model that could
be used to determine how many hours per week each employee should work.

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 57.6 MB/s eta 0:00:00


In [ ]:
import pulp

# --- Model and variables --- #
# indexes #
FT = range(5)   # full-time employees 0..4
PT = range(3)   # part-time 0..2

# Variables #
r = {i: pulp.LpVariable(f"r_{i}", lowBound=0, upBound=40) for i in FT}   # regular hours
o = {i: pulp.LpVariable(f"o_{i}", lowBound=0) for i in FT}               # overtime hours
p = {j: pulp.LpVariable(f"p_{j}", lowBound=0, upBound=20) for j in PT}  # part-time hours

# Deviations #
# Goal 1 #
d1_minus = pulp.LpVariable("d1_minus", lowBound=0)
d1_plus  = pulp.LpVariable("d1_plus",  lowBound=0)

# Goal 2 #
d2_minus = pulp.LpVariable("d2_minus", lowBound=0)
d2_plus  = pulp.LpVariable("d2_plus",  lowBound=0)


# Goal 3 #
d3_minus = pulp.LpVariable("d3_minus", lowBound=0)
d3_plus  = pulp.LpVariable("d3_plus",  lowBound=0)

# Goal 4 #
d4_minus = {i: pulp.LpVariable(f"d4_{i}_minus", lowBound=0) for i in FT}
d4_plus  = {i: pulp.LpVariable(f"d4_{i}_plus",  lowBound=0) for i in FT}

# Helper Expressions (as pulp sums) #
R = 5 * pulp.lpSum((r[i] + o[i]) for i in FT) + 3 * pulp.lpSum(p[j] for j in PT)
W = 6 * pulp.lpSum(r[i] for i in FT) + 10 * pulp.lpSum(o[i] for i in FT) + 3 * pulp.lpSum(p[j] for j in PT)
Profit = 3 * R - W - 500

# Create base problem (we will change objective between stages) #
prob = pulp.LpProblem("Ricky_RecordStore_GoalProg", pulp.LpMinimize)

# --- Goal Equations / Constraints --- #
# Goal 1: Records Target #
prob += R + d1_minus - d1_plus == 1600

# Goal 2: Profit Target #
prob += Profit + d2_minus - d2_plus == 2200

# Goal 3: overtime target (total overtime <= 100) #
prob += pulp.lpSum(o[i] for i in FT) + d3_minus - d3_plus == 100

# Goal 4: each full-time should be 40 hours (minimize shortfall) #
for i in FT:
    prob += r[i] + o[i] + d4_minus[i] - d4_plus[i] == 40

# (Bounds on r, p were set when creating variables) #

# --- Preemptive solving sequence --- #
# Stage 1: minimize d1_minus (sales shortfall) #
prob.setObjective(d1_minus)
prob.solve()
D1_opt = pulp.value(d1_minus)
print("Stage1: d1_minus* =", D1_opt)

# Fix Stage1 #
prob += d1_minus == D1_opt

# Stage 2: minimize d2_minus (profit shortfall) #
prob.setObjective(d2_minus)
prob.solve()
D2_opt = pulp.value(d2_minus)
print("Stage2: d2_minus* =", D2_opt)

# Fix Stage2 #
prob += d2_minus == D2_opt

# Stage 3: minimize d3_plus (overtime excess) #
prob.setObjective(d3_plus)
prob.solve()
D3_opt = pulp.value(d3_plus)
print("Stage3: d3_plus* =", D3_opt)

# Fix Stage3 #
prob += d3_plus == D3_opt

# Stage 4: minimize sum of d4_minus (individual FT shortfalls from 40) #
prob.setObjective(pulp.lpSum(d4_minus[i] for i in FT))
prob.solve()
D4_opt = sum(pulp.value(d4_minus[i]) for i in FT)
print("Stage4: sum d4_minus* =", D4_opt)

# --- Report Solution --- $
print("\nSolution (hours):")
for i in FT:
    print(f" FT {i}: regular={pulp.value(r[i])}, overtime={pulp.value(o[i])}, shortfall_from40={pulp.value(d4_minus[i])}")
for j in PT:
    print(f" PT {j}: hours={pulp.value(p[j])}")

print("\nTotals:")
print(" Records sold R =", pulp.value(R))
print(" Profit =", pulp.value(Profit))
print(" Total overtime =", sum(pulp.value(o[i]) for i in FT))
print(" Deviations: d1_minus,d2_minus,d3_plus =", pulp.value(d1_minus), pulp.value(d2_minus), pulp.value(d3_plus))


Stage1: d1_minus* = 0.0
Stage2: d2_minus* = 0.0
Stage3: d3_plus* = 8.0
Stage4: sum d4_minus* = 0.0

Solution (hours):
 FT 0: regular=40.0, overtime=108.0, shortfall_from40=0.0
 FT 1: regular=40.0, overtime=0.0, shortfall_from40=0.0
 FT 2: regular=40.0, overtime=0.0, shortfall_from40=0.0
 FT 3: regular=40.0, overtime=0.0, shortfall_from40=0.0
 FT 4: regular=40.0, overtime=0.0, shortfall_from40=0.0
 PT 0: hours=20.0
 PT 1: hours=20.0
 PT 2: hours=20.0

Totals:
 Records sold R = 1720.0
 Profit = 2200.0
 Total overtime = 108.0
 Deviations: d1_minus,d2_minus,d3_plus = 0.0 0.0 8.0
